# Part 2 Notebook for the week 3 Coursera Toronto Assignment
## Steps from part 1:
## 1. Install and import
## 2. Scrape the webpage
## 3. Remove records with Borough "not assigned"
## 4. Set "not assigned" Neighborhoods to the name of the Borough
## 5. Combine neighborhoods by Postal Code
## 6. Use the shape command to show the dimensions of the final dataframe
## Steps from part 2
## 7. add lat and long for each postal code.  See section 7 at the bottom of this notebook


### 1. install libraries for webpage scrape and geocoder

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge html5lib --yes
!conda install -c conda-forge requests --yes
!conda install -c conda-forge geocoder --yes
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder # import geocoder

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge

beautifulsoup4 100% |################################| Time: 0:00:00  38.35 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libxml2: 2.9.4-h6b072ca_5     --> 2.9.8-h422b904_2     conda-forge
    libxslt: 1.1.29-hcf9102b_5    --> 1.1.32-h88dbc4e_2    conda-forge
    lxml:    4.1.0-py35ha401a81_0 --> 4.2.5-py35hc9114bc_0 conda-forge

libxml2-2.9.8- 100% |################################| Time: 0:00:00  56.95 MB/s
libxslt-1.1.32 100% |################################| Time: 0:00:00  69.83 MB/s
lxml-4.2.5-py3 100% |################################| Time: 0:00:00  70.48 MB/s
Fetchin

### 2. Scrape the webpage using Beautiful Soup, getting just the table in HTML format
### use Read_HTML to bring the table into a list of dataframes, then pull out the dataframe for the table so that it can be worked with directly
### Change the spelling of the column headers to those assigned by skipping the first row and setting the column headings manually

In [2]:
res = requests.get("http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find('table', attrs={ "class" : "wikitable sortable"})
df = pd.read_html(str(table),skiprows = 1)
boro_DF = df[0]
boro_DF.columns = ['PostalCode', 'Borough', 'Neighborhood']
boro_DF.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 3. Remove Borough 'Not assigned'

In [3]:
boro_DF_assigned = boro_DF[boro_DF.Borough != 'Not assigned']
boro_DF_assigned.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### The assignment shows the dataframe index starting with 0 - reset it

In [4]:
boro_reset_DF = boro_DF_assigned.reset_index(drop=True)
boro_reset_DF.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### 4. Set any 'not assigned' neighborhood to the name of the borough

In [5]:
# define a function that accepts a row, checks the value of neighborhood and assigns the value from the borough column if the neighborhood is "not assigned"
def replace_na_neighborhood(row):
    if row['Neighborhood'] == 'Not assigned':
       row['Neighborhood'] = row['Borough'] 
    return row

In [6]:
# use the replace_na function to update the rows in the dataframe
boro_no_na_DF = boro_reset_DF.apply(replace_na_neighborhood, axis = 1)

In [7]:
boro_no_na_DF.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## 5. Combine Neighborhoods by Postal Code
### In order to step through the dataframe, building the list of neighborhoods by postal code, 
### like postal codes must be together.
### to ensure they are, sort by postal code

In [8]:
sorted_boro_nona_DF = boro_no_na_DF.sort_values('PostalCode')
#sorted_boro_nona_DF.iloc[0:1, :]
sorted_boro_nona_DF.head(12)

,PostalCode,Borough,Neighborhood
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern
23,M1C,Scarborough,Port Union
22,M1C,Scarborough,Rouge Hill
21,M1C,Scarborough,Highland Creek
33,M1E,Scarborough,Guildwood
34,M1E,Scarborough,Morningside
35,M1E,Scarborough,West Hill
39,M1G,Scarborough,Woburn
43,M1H,Scarborough,Cedarbrae


### now all the like postal codes are together
### the following loop simply reads each row of the dataframe, remembering the previous postal code
### if the new postal code is the same as the last one, it concatinates a comma and the new neighborhood name to the 
### saved neighborhood string
### when the postal code changes, it writes out a row to a new dataframe
### finally, when the loop ends, there's still data in the "save" fields, so that last new row is written

In [9]:
# rather than slow the following code down by making it check for the initial values on each iteration, 
# I decided to just go ahead and write a bad record the first time through, then remove it later.
# I also ruled out using the first row of the input as the initialization because I would have to add a condition 
# to make sure the first neighborhood was not repeated, and that would have to be checked on each iteration as well
# so, this code purposely writes ["A", "", ""] as the first output record, then moves on with less code.

output_DF = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
save_pc = 'A'
save_nb = ''
save_bo = ''
for index, row in sorted_boro_nona_DF.iterrows():
    if row['PostalCode'] == save_pc:
        save_nb = save_nb + ', ' +row["Neighborhood"]
    else:
        output_DF = output_DF.append({
             "PostalCode": save_pc,
             "Borough":  save_bo,
             "Neighborhood": save_nb
              }, ignore_index=True)
        save_pc = row['PostalCode']
        save_bo = row['Borough']
        save_nb = row['Neighborhood']  
        
#write the final row out to the output dataframe
output_DF = output_DF.append({
             "PostalCode": save_pc,
             "Borough":  save_bo,
             "Neighborhood": save_nb
              }, ignore_index=True)

#now drop the first row ("A", "", "") while assigning the DF to a new usable name
compact_DF = output_DF.drop([0])
# and make the index column sensible
final_compact_DF = compact_DF.reset_index(drop=True)
    

In [10]:
final_compact_DF

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea"
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


### 6. instructions say to make the shape command the last cell of the workbook:

In [11]:
final_compact_DF.shape

(103, 3)

### 7. get lat and long for each postal code

In [13]:
output_LL_DF = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])
for index, row in final_compact_DF.iterrows():
    lat_lng_coords = None
# loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(row['PostalCode']))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    output_LL_DF = output_LL_DF.append({
             "PostalCode": row['PostalCode'],
             "Borough":  row['Borough'],
             "Neighborhood": row['Neighborhood'], 
             "Latitude": latitude,
             "Longitude": longitude
              }, ignore_index=True)



output_LL_DF

KeyboardInterrupt: 

## since geocoder is giving no response, use the CSV file

In [14]:
!wget -q -O 'postalcode_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [30]:
postal_df = pd.read_csv('postalcode_data.csv').set_index('Postal Code')
postal_df

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


In [36]:
output_LL_DF = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])
for index, row in final_compact_DF.iterrows():
    current_Postalrow = postal_df.loc[ row['PostalCode'] , : ]
    latitude = current_Postalrow[0]
    longitude = current_Postalrow[1]
    output_LL_DF = output_LL_DF.append({
             "PostalCode": row['PostalCode'],
             "Borough":  row['Borough'],
             "Neighborhood": row['Neighborhood'], 
             "Latitude": latitude,
             "Longitude": longitude
              }, ignore_index=True)



output_LL_DF

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


In [37]:
output_LL_DF.shape

(103, 5)